# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import pickle

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [16]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM messages", engine)
X = df['message'].values 
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
categories = y.columns
y = y.values
categories

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [6]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [7]:
def tokenize(text):
    '''Normalize, tokenize and lemmatize a text string
    
    Args:
        text (str): text to tokenize
    
    Returns:
        tokenized text
    '''
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
# Split the data into train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
y_pred = pipeline.predict(X_test)

for i in range(len(categories)):
    print('Label:', categories[i])
    print(classification_report(y_test[:, i], y_pred[:, i]))

Label: related
             precision    recall  f1-score   support

          0       0.63      0.36      0.46      1500
          1       0.83      0.94      0.88      4998
          2       0.52      0.51      0.52        47

avg / total       0.78      0.80      0.78      6545

Label: request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5406
          1       0.83      0.35      0.49      1139

avg / total       0.87      0.87      0.85      6545

Label: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6514
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6545

Label: aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.79      3810
          1       0.76      0.54      0.63      2735

avg / total       0.74      0.74      0.73      6545

Label: medical_help
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
parameters = {
    'vect__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': [True, False],
    'clf__estimator__min_samples_split': [2, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)

for i in range(len(categories)):
    print('Label:', categories[i])
    print(classification_report(y_test[:, i], y_pred_cv[:, i]))

Label: related
             precision    recall  f1-score   support

          0       0.60      0.39      0.47      1500
          1       0.83      0.92      0.87      4998
          2       0.41      0.40      0.41        47

avg / total       0.77      0.79      0.78      6545

Label: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5406
          1       0.83      0.38      0.52      1139

avg / total       0.87      0.88      0.86      6545

Label: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6514
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6545

Label: aid_related
             precision    recall  f1-score   support

          0       0.71      0.89      0.79      3810
          1       0.76      0.50      0.60      2735

avg / total       0.73      0.73      0.71      6545

Label: medical_help
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [30]:
pickle_out = open('model.pickle', 'wb')
pickle.dump(cv, pickle_out)
pickle_out.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.